In [ ]:
import numpy as np
import scipy
import scipy.linalg as la
import functools as ft
import matplotlib.pyplot as plt
import tinyarray as ta
import kwant
import matplotlib.pyplot as plt
import pickle

from hamiltonians import SnTe_18band_disorder, site_type, SnPbTe_params
from spectral_functions import surface_dos

# Calculate surface spectral functions of disordered slabs with PBC in x and y

In [ ]:
import hpc05

In [ ]:
hpc05.kill_remote_ipcluster()

In [ ]:
client, dview, lview = hpc05.start_remote_and_connect(11, profile='pbs_32GB', timeout=600,
                                                      env_path='~/.conda/envs/kwant_dev',
                                                      folder='~/disorder_invariants/code/',
                                                     )

In [ ]:
%%px --local
import itertools as it
import scipy
import scipy.linalg as la
import numpy as np
import copy
import functools as ft
import pickle
import tinyarray as ta

import kwant
from hamiltonians import SnTe_18band_disorder, site_type, SnPbTe_params
from spectral_functions import surface_dos

In [ ]:
%%px --local
salt = '0'

# Make a slab with PBC in x and y
Lx = Ly = 80
Lz = 20
Ls = ta.array([Lx, Ly, Lz])

num_moments = 4000
energies = np.linspace(-1, 1, 1001)

d = 1
# surface layer
window = lambda pos: pos[2] >= Ls[2]/2 - d + 0.5
# slab
shape = lambda site: -Ls[2]/2 < site.pos[2] <= Ls[2]/2

@ft.lru_cache(maxsize=1)
def make_syst(Ls, doping, salt):
    Lx, Ly, Lz = Ls
    syst = SnTe_18band_disorder()

    # Build film using syst
    film = kwant.Builder(kwant.lattice.TranslationalSymmetry([Lx, 0, 0], [0, Ly, 0]))

    film.fill(syst, shape, start=np.zeros(3));
    filmw = kwant.wraparound.wraparound(film)   
    filmw = filmw.finalized()

    pars = SnPbTe_params.copy()

    pars['site_type'] = ft.partial(site_type, doping=doping, n=None, to_fd=None, salt=salt)
    pars['k_x'] = pars['k_y'] = pars['k_z'] = 0
    
    return filmw, pars

def job(k, n=ta.array([1, 1, 0])/np.sqrt(2)):
    filmw, pars = make_syst(Ls, doping, salt)
    print(k, end='\r')
    
    sf = surface_dos(filmw,
                     k * n,
                     pars,
                     supercell=np.diag(Ls),
                     pos_transform=None,
                     num_moments=num_moments,
                     bounds=None,
                     window=window)
    return sf(energies)

In [ ]:
%%px --local
doping = 0.8

In [ ]:
# %%time
ks = np.linspace(1.5, np.pi/np.sqrt(2), 101)
result = lview.map_async(job, ks)

In [ ]:
result.wait_interactive()

In [ ]:
res = result.get()
spec_func = np.array(res).T.real
plt.figure(figsize=(20,10))
plt.imshow(spec_func,
           # vmin=0, vmax=200,
           extent=(ks[0], ks[-1], energies[0], energies[-1]),
           # aspect=1/2, 
           origin='lower',
           
           )
plt.colorbar()

In [ ]:
pickle.dump(dict(
                doping=doping,
                spec_func=spec_func,
                Lx=Lx,
                Ly=Ly,
                Lz=Lz,
                salt=salt,
                num_moments=num_moments,
                ),
            open('../data/SnTe_18_orb_specfunc_'+str(Lx)+'x'+str(Ly)+'x'+str(Lz)+'_'+str(doping)+'.pickle', 'wb'))